# set up packages

In [16]:
# load packages
import numpy as np
import pandas as pd
import os
os.environ["PYSPARK_PYTHON"] = "python3"

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("moive analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
location = "D:/py_movie_recommendation_system/data/"
movies_df = spark.read.load(location+"movies.csv", format='csv', header = True)
ratings_df = spark.read.load(location+"ratings.csv", format='csv', header = True)
links_df = spark.read.load(location+"links.csv", format='csv', header = True)
tags_df = spark.read.load(location+"tags.csv", format='csv', header = True)

# data preprocessing

In [4]:
# drop useless column
movie_ratings=ratings_df.drop('timestamp')

In [5]:
# Data type convert
from pyspark.sql.types import IntegerType, FloatType
movie_ratings = movie_ratings.withColumn("userId", movie_ratings["userId"].cast(IntegerType()))
movie_ratings = movie_ratings.withColumn("movieId", movie_ratings["movieId"].cast(IntegerType()))
movie_ratings = movie_ratings.withColumn("rating", movie_ratings["rating"].cast(FloatType()))

In [6]:
movie_ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
|     1|    101|   5.0|
|     1|    110|   4.0|
|     1|    151|   5.0|
|     1|    157|   5.0|
|     1|    163|   5.0|
|     1|    216|   5.0|
|     1|    223|   3.0|
|     1|    231|   5.0|
|     1|    235|   4.0|
|     1|    260|   5.0|
|     1|    296|   3.0|
|     1|    316|   3.0|
|     1|    333|   5.0|
|     1|    349|   4.0|
+------+-------+------+
only showing top 20 rows



# train test split

In [7]:
# import package
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder

In [8]:
# Create test and train set
(training,test) = movie_ratings.randomSplit([0.8,0.2])

# tune model

In [9]:
# Create ALS model
model_als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop", seed=202112)

In [10]:
# Define evaluator as RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

In [11]:
# Tune model using ParamGridBuilder
params = ParamGridBuilder().addGrid(model_als.maxIter, [3, 5, 10]).addGrid(model_als.regParam, [0.1, 0.01, 0.001]).addGrid(model_als.rank, [5, 10, 15]).addGrid(model_als.alpha, [0.1, 0.01, 0.001]).build()

# Build Cross validation
cv_creator=CrossValidator(estimator=model_als,estimatorParamMaps=params,evaluator=evaluator,numFolds=4,seed=202112)

#Fit ALS model to training data
cv_model=cv_creator.fit(training)

In [12]:
best_model=cv_model.bestModel
best_params=cv_model.getEstimatorParamMaps()[np.argmin(cv_model.avgMetrics)]
print('Best ALS model parameters by CV:')
for i,j in best_params.items():
  print('-> '+i.name+': '+str(j))

Best ALS model parameters by CV:
-> maxIter: 10
-> regParam: 0.1
-> rank: 5
-> alpha: 0.1


# model tesing

In [13]:
#Generate predictions and evaluate using RMSE
prediction_test = best_model.transform(test)
rmse_test = evaluator.evaluate(prediction_test)
print("Root-mean-square error for testing data is " + str(rmse_test))

Root-mean-square error for testing data is 0.8861625353823233


# qualitative check

In [48]:
# define a function to package the recommendation
def top_K_recommend(k, uid, model, data_location="D:/py_movie_recommendation_system/data/"):
    """
    k: the number of movies to recommend
    id: the id of the user to give recommendations
    model: the trained model for recommendation
    """
    # the table for all top10 recommendations
    from pyspark.sql import SparkSession
    spark = SparkSession \
        .builder \
        .appName("moive analysis") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    movies_df = spark.read.load(data_location + "movies.csv", format='csv', header=True)

    all_recommend = model.recommendForAllUsers(k)
    user_recommend = all_recommend.where(all_recommend.userId == uid).toPandas()
    if user_recommend.shape[0] == 0:
        print('No user with id ' + str(uid) + ' is found in the data.')
        print("Would you like to watch most frequently watched movie?")
        ratings_df = spark.read.load(data_location + "ratings.csv", format='csv', header=True)
        movies_df.registerTempTable("movies")
        ratings_df.registerTempTable("ratings")
        out = spark.sql(
            f"""
            SELECT a.movieId, a.title, a.genres
            FROM movies AS a
            LEFT JOIN (
            SELECT movieId, COUNT(rating) AS rating
            FROM ratings
            GROUP BY movieId
            ) AS b
            USING(movieId)
            ORDER BY b.rating DESC
            LIMIT {k}
            """
        ).toPandas()
        return out
    user_recommend = user_recommend.iloc[0, 1]
    user_recommend = pd.DataFrame(user_recommend, columns=['movieId', 'predicted_ratings'])
    temp = None
    for i in user_recommend['movieId']:
        if not temp:
            temp = movies_df.where(movies_df.movieId == str(i))
        else:
            temp = temp.union(movies_df.where(movies_df.movieId == str(i)))
    out = pd.concat([temp.toPandas(), user_recommend['predicted_ratings']], axis=1)
    out.index = range(1, k + 1)
    return out


# cosine similarity
# the larger the cosine value, the smaller the two feature vectors' angle, the similar the movies
# this similarity considers the direction only,
# e.g. movie 1 with factor [1,2,3] and movie 2 with factor [2,4,6] are considered the same
def cos_similar(k, mid, best_model, data_location="D:/py_movie_recommendation_system/data/"):
    """
    k: number of similar movies to find
    mid: id of the movie to find similarities
    """
    from pyspark.sql import SparkSession
    spark = SparkSession \
        .builder \
        .appName("moive analysis") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    movies_df = spark.read.load(data_location + "movies.csv", format='csv', header=True)
    movies_df.registerTempTable("movies")

    movie_factors = best_model.itemFactors
    movie_factors.printSchema()
    comd = ["movie_factors.selectExpr('id as movieId',"]
    for i in range(best_model.rank):
        if i < best_model.rank - 1:
            comd.append("'features[" + str(i) + "] as feature" + str(i) + "',")
        else:
            comd.append("'features[" + str(i) + "] as feature" + str(i) + "'")
    comd.append(')')
    movie_factors = eval(''.join(comd))
    movie_factors.createOrReplaceTempView('movie_factors')

    movie_info = spark.sql(
        f"""
        SELECT *
        FROM movie_factors
        WHERE movieId= {mid}
        """
        ).toPandas()
    if movie_info.shape[0] <= 0:
        print('No movie with id ' + str(mid) + ' is found in the data.')
        return None, None
    norm_m = sum(movie_info.iloc[0, 1:].values ** 2) ** 0.5
    temp = ['select movieId,']
    norm_str = ['sqrt(']
    for i in range(best_model.rank):
        comd = 'feature' + str(i) + '*' + str(movie_info.iloc[0, i + 1])
        temp.append(comd + ' as inner' + str(i) + ',')
        if i < best_model.rank - 1:
            norm_str.append('feature' + str(i) + '*feature' + str(i) + '+')
        else:
            norm_str.append('feature' + str(i) + '*feature' + str(i))
    norm_str.append(') as norm')
    temp.append(''.join(norm_str))
    temp.append(' from movie_factors where movieId!=' + str(mid))
    inner = spark.sql(' '.join(temp))
    inner = inner.selectExpr('movieId',
                             '(inner0+inner1+inner2+inner3+inner4)/norm/' + str(norm_m) + ' as innerP').orderBy(
        'innerP', ascending=False).limit(k).toPandas()
    out = None
    for i in inner['movieId']:
        if not out:
            out = movies_df.where(movies_df.movieId == str(i))
        else:
            out = out.union(movies_df.where(movies_df.movieId == str(i)))
    out = out.toPandas()
    out.index = range(1, k + 1)
    return out, inner



# write a function to make prediction for movie id
def similar_movie(k, mid, best_model, data_location="D:/py_movie_recommendation_system/data/"):
    out, inner = cos_similar(k, mid, best_model, data_location)
    print(out)
    if out is None:
        print("This is a new movie, and we cannot find similar movie only based on it id!")
    return out

# write a function to make recommendation
def make_recommendation(k, uid, best_model, data_location="D:/py_movie_recommendation_system/data/"):
    out = top_K_recommend(k, uid, best_model, data_location)
    print(out)
    return out

## for user

In [49]:
# user in dataset
out = make_recommendation(10, 500, best_model)

c:\programming\python\lib\site-packages\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


   movieId                                          title  \
1    84847                                    Emma (2009)   
2     2693                                Trekkies (1997)   
3    72171                          Black Dynamite (2009)   
4     5485                                 Tadpole (2002)   
5   158783                          The Handmaiden (2016)   
6      446  Farewell My Concubine (Ba wang bie ji) (1993)   
7    25771  Andalusian Dog, An (Chien andalou, Un) (1929)   
8    26865     Fist of Legend (Jing wu ying xiong) (1994)   
9     8477                               Jetée, La (1962)   
10    7116            Diabolique (Les diaboliques) (1955)   

                     genres  predicted_ratings  
1      Comedy|Drama|Romance           5.485587  
2               Documentary           5.298022  
3             Action|Comedy           5.282207  
4      Comedy|Drama|Romance           5.236969  
5    Drama|Romance|Thriller           5.147257  
6             Drama|Romance       

In [50]:
spark.sql(
    """
    SELECT *
    FROM ratings as r
    LEFT JOIN movies as m
    USING(movieId)
    WHERE r.userId = 500
    ORDER BY r.rating DESC
    """
).toPandas()

,movieId,userId,rating,timestamp,title,genres
0,176,500,5.0,1005527755,Living in Oblivion (1995),Comedy
1,1175,500,5.0,1005528017,Delicatessen (1991),Comedy|Drama|Romance
2,1282,500,5.0,1005528236,Fantasia (1940),Animation|Children|Fantasy|Musical
3,1747,500,5.0,1005528065,Wag the Dog (1997),Comedy
4,1784,500,5.0,1005527784,As Good as It Gets (1997),Comedy|Drama|Romance
...,...,...,...,...,...,...
81,1993,500,1.0,1005527364,Child's Play 3 (1991),Comedy|Horror|Thriller
82,2671,500,1.0,1005528078,Notting Hill (1999),Comedy|Romance
83,3083,500,1.0,1005528017,All About My Mother (Todo sobre mi madre) (1999),Drama
84,3466,500,1.0,1005527926,Heart and Souls (1993),Comedy|Fantasy


In [51]:
out = make_recommendation(10, 1111111111111111, best_model)

c:\programming\python\lib\site-packages\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
c:\programming\python\lib\site-packages\pyspark\sql\dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


No user with id 1111111111111111 is found in the data.
Would you like to watch most frequently watched movie?
  movieId                                      title  \
0     356                        Forrest Gump (1994)   
1     318           Shawshank Redemption, The (1994)   
2     296                        Pulp Fiction (1994)   
3     593           Silence of the Lambs, The (1991)   
4    2571                         Matrix, The (1999)   
5     260  Star Wars: Episode IV - A New Hope (1977)   
6     480                       Jurassic Park (1993)   
7     110                          Braveheart (1995)   
8     589          Terminator 2: Judgment Day (1991)   
9     527                    Schindler's List (1993)   

                             genres  
0          Comedy|Drama|Romance|War  
1                       Crime|Drama  
2       Comedy|Crime|Drama|Thriller  
3             Crime|Horror|Thriller  
4            Action|Sci-Fi|Thriller  
5           Action|Adventure|Sci-Fi  
6  Acti

## for movie

In [52]:
# movie in the dataset
out = similar_movie(10, 858, best_model)

root
 |-- id: integer (nullable = false)
 |-- features: array (nullable = true)
 |    |-- element: float (containsNull = false)



c:\programming\python\lib\site-packages\pyspark\sql\dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(
c:\programming\python\lib\site-packages\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


   movieId                                  title                 genres
1     2311  2010: The Year We Make Contact (1984)                 Sci-Fi
2     1221         Godfather: Part II, The (1974)            Crime|Drama
3     3430                      Death Wish (1974)     Action|Crime|Drama
4     1653                         Gattaca (1997)  Drama|Sci-Fi|Thriller
5      357     Four Weddings and a Funeral (1994)         Comedy|Romance
6     2333               Gods and Monsters (1998)                  Drama
7    30848     Love Song for Bobby Long, A (2004)                  Drama
8      936                       Ninotchka (1939)         Comedy|Romance
9     3095            Grapes of Wrath, The (1940)                  Drama
10    1945               On the Waterfront (1954)            Crime|Drama


In [53]:
# movie not in dataset
out = similar_movie(10, 1111111, best_model)

root
 |-- id: integer (nullable = false)
 |-- features: array (nullable = true)
 |    |-- element: float (containsNull = false)

No movie with id 1111111 is found in the data.
None
This is a new movie, and we cannot find similar movie only based on it id!


c:\programming\python\lib\site-packages\pyspark\sql\dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(
c:\programming\python\lib\site-packages\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


# save the model

In [54]:
best_model.save("D:/py_movie_recommendation_system/spark_warehouse")

Py4JJavaError: An error occurred while calling o71068.save.
: java.lang.RuntimeException: java.io.FileNotFoundException: Could not locate Hadoop executable: C:\Programming\hadoop-3.3.1\bin\winutils.exe -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:736)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:271)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:287)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSy